In [1]:
! pip install mesa --quiet
#! pip install flask --quiet

In [2]:
# Librerías de simulación multiagentes
from mesa import Agent, Model  # Define los agentes y el modelo base de la simulación
from mesa.space import MultiGrid, SingleGrid  # Permite colocar los agentes en una cuadrícula (multi o individual)
from mesa.time import SimultaneousActivation  # Activa a todos los agentes simultáneamente en cada paso de la simulación
from mesa.time import BaseScheduler
from mesa.datacollection import DataCollector  # Recolecta y organiza datos de la simulación para análisis

# Librerías matemáticas y generación de aleatoriedad
import itertools  # Proporciona herramientas para crear combinaciones y permutaciones
import random  # Permite generar números y secuencias aleatorias, útil para la variabilidad en simulaciones
import math  # Contiene funciones matemáticas básicas, como operaciones trigonométricas y logarítmicas

# Librerías de visualización y gráficos
import matplotlib.pyplot as plt  # Creación y personalización de gráficos
import matplotlib.patches as patches  # Añade formas como rectángulos o círculos a gráficos de matplotlib
from matplotlib.colors import ListedColormap  # Define mapas de color personalizados
import matplotlib.animation as animation  # Crea animaciones, útil para representar dinámicamente la simulación
plt.rcParams["animation.html"] = "jshtml"  # Configura la visualización de animaciones en formato HTML
import matplotlib  # Configuración adicional de matplotlib
matplotlib.rcParams["animation.embed_limit"] = 2**128  # Ajusta el límite de tamaño para incrustar animaciones

# Librerías de manipulación y análisis de datos
import numpy as np  # Proporciona funciones para manejo de matrices y álgebra lineal, común en análisis de datos
import pandas as pd  # Ofrece estructuras de datos como DataFrames, útiles para manipular grandes cantidades de datos
import time  # Proporciona funciones para trabajar con fechas y horas

# Librerías de comunicación en red y servidor HTTP
from http.server import BaseHTTPRequestHandler, HTTPServer  # Implementa un servidor HTTP básico para manejar solicitudes
import logging  # Registro de eventos y errores del servidor, útil para el diagnóstico
import json  # Proporciona funciones para manejar datos en formato JSON, útil para comunicación entre aplicaciones

import heapq
from queue import Queue

In [28]:
class LuigiAgent(Agent):
    """Agente que simula a Luigi en el modelo de rescate y bombero."""

    def __init__(self, unique_id, model, role):
        super().__init__(unique_id, model)
        self.role = role  # rol: 'rescuer' o 'firefighter'
        self.pos = None  # Posición del agente en el grid
        self.model = model
        self.history = []  # Histórico de movimientos
        self.action_points = 4  # Puntos de acción para cada turno
        self.carrying_portrait = False  # Indica si el agente lleva un retrato (víctima)

    def manhattan(self, target):
        """Calcula la distancia Manhattan entre el agente y el objetivo."""
        if not isinstance(target, tuple) or len(target) != 2:
            raise ValueError(f"Target inválido para cálculo de Manhattan: {target}")
        x, y = self.pos
        tx, ty = target
        return abs(x - tx) + abs(y - ty)

    def move_towards(self, target):
        """Mueve al agente hacia una celda objetivo usando la distancia Manhattan."""
        if self.pos == target:
            return True  # El agente ya está en la celda objetivo
        
        def get_adjacent_positions(pos):
            """Devuelve posiciones adyacentes válidas alrededor de `pos`."""
            x, y = pos
            possible_moves = [
                (x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)
            ]

            # Filtrar solo las posiciones dentro de los límites del grid
            return [
                (nx, ny) for nx, ny in possible_moves
                if 0 <= nx < self.model.grid.width and 0 <= ny < self.model.grid.height and self.model.grid.is_cell_empty((nx, ny))
            ]

        # Implementar búsqueda en amplitud (BFS) para encontrar un camino
        visited = set()
        queue = Queue()
        queue.put((self.pos, []))  # (posición actual, camino hasta ahora)

        while not queue.empty():
            current_pos, path = queue.get()
            
            if current_pos in visited:
                continue
            
            visited.add(current_pos)

            for next_pos in get_adjacent_positions(current_pos):
                if next_pos == target:
                    path.append(next_pos)
                    break  # Encontramos el objetivo

                if next_pos not in visited:
                    queue.put((next_pos, path + [next_pos]))
            
            if path and path[-1] == target:
                break  # Finalizar la búsqueda cuando se encuentre un camino al objetivo

        # Verificar si el agente tiene suficiente energía para moverse
        if self.carrying_portrait and self.action_points < 2:
            print(f"Agente {self.unique_id} no tiene suficientes puntos de acción para moverse mientras carga un retrato.")
            return False  # No hay suficientes puntos para moverse con el retrato

        elif not self.carrying_portrait and self.action_points < 1:
            print(f"Agente {self.unique_id} no tiene suficientes puntos de acción para moverse.")
            return False  # No hay suficientes puntos para moverse sin el retrato

        # Si se encontró un camino, moverse hacia el primer paso del camino
        if path:
            next_step = path[0]
            print(f"Agente {self.unique_id} se mueve de {self.pos} a {next_step}.")
            self.model.grid.move_agent(self, next_step)
            self.pos = next_step
            self.history.append(self.pos)

            if self.carrying_portrait:
                self.action_points -= 2
            else: 
                self.action_points -= 1
            return True
        else:
            print(f"Agente {self.unique_id} no puede alcanzar el objetivo desde {self.pos}.")
            return False  # No se encontró un camino válido

    def examine_portrait(self, position):
        """Recoge el retrato de la víctima cuando el agente llega a su posición.
        Examina el retrato y determina si es una víctima o una falsa alarma.
        """
        if position in self.model.portraits:
            portrait = self.model.portraits[position]
            if portrait == "victim":
                self.carrying_portrait = True  # El agente ahora lleva un retrato (víctima)
                #self.model.saved_count += 1
                self.model.portraits[position] = None  # Eliminar la víctima rescatada
                print(f"Agente {self.unique_id} ha encontrado una víctima en {position}.")
            elif portrait == "false_alarm":
                print(f"Agente {self.unique_id} encontró una falsa alarma en {position}. Eliminando de la lista y buscando otro retrato.")
                self.model.portraits[position] = None  # Eliminar la falsa alarma
                return False  # Indicar que debe buscar otro retrato
        return True

    def extinguish_fire(self, position):
        """Extinguir fuego en la posición dada."""
        if self.action_points >= 2:
            print(f"[DEBUG] Agente {self.unique_id} apagando fuego en {position}.")
            self.model.grid_details[position] = 0  # Actualizar la celda a estado vacío
            self.action_points -= 2  # Reducir puntos de acción
        else:
            print(f"[DEBUG] Agente {self.unique_id} no tiene suficientes puntos para apagar fuego en {position}.")

    def extinguish_smoke(self, position):
        """Extinguir humo en la posición dada."""
        if self.action_points >= 1:
            print(f"[DEBUG] Agente {self.unique_id} eliminando humo en {position}.")
            self.model.grid_details[position] -= 1   # Actualizar la celda a estado vacío
            self.action_points -= 1  # Reducir puntos de acción
        else:
            print(f"[DEBUG] Agente {self.unique_id} no tiene suficientes puntos para apagar humo en {position}.")

    def rescuer_strategy(self):
        """Estrategia para agentes cuyo rol es rescatar víctimas."""
        while self.action_points > 0:
            if self.carrying_portrait:
                # Si lleva un retrato, moverse hacia la salida más cercana
                valid_exits = [pos for pos in self.model.entrances if isinstance(pos, tuple) and len(pos) == 2]
                if valid_exits:
                    nearest_exit = min(valid_exits, key=self.manhattan)
                    print(f"[DEBUG] Agente {self.unique_id} lleva retrato. Moviéndose hacia la salida más cercana: {nearest_exit}")
                    if not self.move_towards(nearest_exit):  # Detenerse si no puede moverse
                        print(f"[DEBUG] Agente {self.unique_id} no puede moverse hacia {nearest_exit}.")
                        break

                     # Si el agente llega a la salida, se rescató la víctima
                    if self.pos == nearest_exit:
                        print(f"[DEBUG] Agente {self.unique_id} ha llegado a la salida con el retrato.")
                        self.carrying_portrait = False  # Resetear estatus de
                        self.model.rescued += 1  # Contar el rescate en el modelo
                        print(f"[DEBUG] Agente {self.unique_id} ha rescatado a una víctima. Total rescatados: {self.model.rescued}")

                else:
                    print(f"[ERROR] No hay salidas válidas para el agente {self.unique_id}. Terminando turno.")
                    break 
            else:
                # Buscar el retrato más cercano (víctima o falsa alarma)
                portraits = [
                    pos for pos, label in self.model.portraits.items()
                    if label in ["victim", "false_alarm"] and isinstance(pos, tuple) and len(pos) == 2
                ]
                if portraits:
                    # Encontrar el retrato más cercano usando Manhattan
                    nearest_portrait = min(portraits, key=self.manhattan)
                    print(f"[DEBUG] Agente {self.unique_id} buscando retrato. Moviéndose hacia el retrato más cercano: {nearest_portrait}")
                    if not self.move_towards(nearest_portrait):  # Si no puede moverse, detenerse
                        break
                    # Verificar si llegó al retrato y procesarlo
                    if self.pos == nearest_portrait:
                        if not self.examine_portrait(nearest_portrait):  # Si no es una víctima, sigue buscando otro retrato
                            continue
                else:
                    print(f"[DEBUG] No hay más retratos para el agente {self.unique_id}. Terminando turno.")
                    break

    def firefighter_strategy(self):
        """Estrategia para agentes cuyo rol es apagar incendios."""
        while self.action_points > 0:
            # Buscar la celda más cercana con humo (valor 1) o fuego (valor 2)
            fire_cells = [pos for pos, value in self.model.grid_details.items() if value == 1 or value == 2]
            if fire_cells:
                nearest_fire = min(fire_cells, key=self.manhattan)
                print(f"[DEBUG] Agente {self.unique_id} buscando fuego. Moviéndose hacia el fuego más cercano: {nearest_fire}")
                if not self.move_towards(nearest_fire):  # Si no puede moverse hacia el fuego, detenerse
                    break
                # Verificar si llegó al humo/fuego y procesarlo
                if self.pos == nearest_fire:
                    fire_value = self.model.grid_details[nearest_fire]
                    if fire_value == 2: # Si es fuego
                        # Dependiendo de puntos, extinguir o reducir a humo
                        if self.action_points >= 2:
                            self.extinguish_fire(nearest_fire)
                        elif self.action_points == 1:
                            self.extinguish_smoke(nearest_fire)
                    elif fire_value == 1: # Si tiene puntos y es humo
                        self.extinguish_smoke(nearest_fire)     
                    continue # Después de lidiar con el humo/fuego, sigue buscando otro
            else:
                # Si no hay fuego, el bombero se detiene
                print(f"[DEBUG] Agente {self.unique_id} no encuentra más fuego ni humo.")
                break

    def step(self):
        """Función que ejecuta el paso de un agente."""
        print(f"\n[DEBUG] Agente {self.unique_id} ({self.role}) inicia su turno en posición {self.pos}. Energía inicial: {self.action_points}.")
        
        if self.role == "rescuer":
            self.rescuer_strategy()  # Ejecutar estrategia de rescate
        elif self.role == "firefighter":
            self.firefighter_strategy()  # Ejecutar estrategia de apagar incendios

        print(f"[DEBUG] Agente {self.unique_id} ({self.role}) termina su turno en posición {self.pos}. Energía restante: {self.action_points}.")
        
        # Restaurar la energía para el próximo turno
        self.action_points += 4


    def toggle_point(self):
        """Método que permite al agente alternar entre llevar un retrato o no."""
        self.carrying_portrait = not self.carrying_portrait
        print(f"Agente {self.unique_id} ahora {'lleva un retrato' if self.carrying_portrait else 'no lleva un retrato'}.")



In [68]:
class MansionModel(Model):
    def __init__(self, luigis, fake_alarms, victims, walls, doors, boo, entrances):
        # Inicializar la clase base Model sin argumentos adicionales
        super().__init__()

        # Variables iniciales del modelo
        self.step_count = 0
        self.schedule = BaseScheduler(self)  # Usamos BaseScheduler para compatibilidad futura
        self.rescued = 0
        self.losses = 0
        self.casualties = 0
        #self.saved_count = 0
        self.simulation_status = "In progress"
        self.boo_zones = [(col - 1, row - 1) for row, col in boo]
        self.wall_config = walls

        # Configuración del recolector de datos
        self.datacollector = DataCollector(
            model_reporters={
                "Grid": "get_grid",
                "Walls": "get_walls",
                "Steps": "step_count",
                "Doors": "doors",
                "Damage": "damage_counter",
                "Status": "simulation_status",
                "Portraits": "portraits",
                "Rescued": "rescued",
                "Losses": "losses"
            },
            agent_reporters={
                "Agent_ID": lambda agent: agent.unique_id,
                "Role": lambda agent: agent.role,
                "Position": lambda agent: (agent.pos[0], agent.pos[1]),
                "Agent History": lambda agent: getattr(agent, 'history', [])
            }
        )

        self.portraits = {}
        for (row, col) in fake_alarms:
            self.portraits[(int(col), int(row))] = "false_alarm"
        for (row, col) in victims:
            self.portraits[(int(col), int(row))] = "victim"

        # Dimensiones del grid
        self.grid_width = 9
        self.grid_height = 7

        # Configuración de puertas y entradas
        self.exit_positions = doors
        self.entrances = [(int(col), int(row)) for row, col in entrances]

        # Crear el espacio y los detalles del grid
        self.grid = MultiGrid(self.grid_width, self.grid_height, torus=False)
        self.grid_details = {(x, y): 0 for y in range(self.grid_height) for x in range(self.grid_width)}
        self.grid_walls = {(x, y): ["0000", "0000"] for y in range(self.grid_height) for x in range(self.grid_width)}
        self.damage_counter = 0

        # Configurar los muros
        for y, row in enumerate(self.wall_config):
            for x, value in enumerate(row):
                self.grid_walls[(x, y)] = [value, "0000"]

        # Configurar zonas de fantasmas
        for position in self.boo_zones:
            self.grid_details[position] = 2

        # Agregar agentes Luigi con roles alternados
        agent_roles = ["rescuer", "firefighter"]
        agent_role_cycle = itertools.cycle(agent_roles)
        agent_cy = itertools.cycle(self.entrances)
        for idx in range(luigis):
            position = next(agent_cy)
            role = next(agent_role_cycle)
            agent = LuigiAgent(idx, self, role)
            agent.unique_id = idx
            self.grid.place_agent(agent, position)
            self.schedule.add(agent)
            print(f"Agente {idx} con rol {role} colocado en posición {position}")

    def add_portraits(self):
        """Agrega retratos si hay menos de 3 activos."""
        active_points = sum(1 for portrait in self.portraits.values() if portrait in ["victim", "false_alarm"])
        
        if active_points < 3:
            needed_points = 3 - active_points
            new_points = 0

            while new_points < needed_points:
                candidate_point = (random.randint(0, self.grid_width - 1), random.randint(0, self.grid_height - 1))
                if candidate_point not in self.portraits:
                    if self.grid_details.get(candidate_point) in [1, 2]:  # 1 for smoke, 2 for fire
                        # Eliminar fuego o humo y colocar el retrato en su lugar
                        self.grid_details[candidate_point] = 0  # Remove smoke/fire (set to 0)
                        print(f"[DEBUG] El fuego/humo en {candidate_point} fue removido para poner un retrato.")

                    # Agregar un nuevo retrato (víctima o falsa alarma)
                    if random.choice([True, False]):
                        self.portraits[candidate_point] = "victim"
                    else:
                        self.portraits[candidate_point] = "false_alarm"

                    self.grid_details[candidate_point] = 0
                    new_points += 1
                    print(f"[DEBUG] Se agregó un nuevo retrato en {candidate_point} tipo: ({self.portraits[candidate_point]})")
                else:
                    agents_cell = self.grid.get_cell_list_contents([candidate_point])
                    for agent in agents_cell:
                        if isinstance(agent, LuigiAgent):
                            agent.toggle_point()
                            break

    def spread_boos(self):
        """Extiende la presencia de fantasmas en el mapa."""
        affected_positions = [pos for pos, val in self.grid_details.items() if val in (0, 1, 2)]
        if affected_positions:
            target_pos = random.choice(affected_positions)
            if self.grid_details[target_pos] == 0:
                self.grid_details[target_pos] = 1
            elif self.grid_details[target_pos] == 1:
                self.grid_details[target_pos] = 2
            elif self.grid_details[target_pos] == 2:
                neighbors = self.grid.get_neighborhood(target_pos, moore=False, include_center=False)
                for neighbor in neighbors:
                    if self.check_collision(target_pos, neighbor):
                        self.register_damage(target_pos, neighbor)
                    if self.grid_details[neighbor] == 0:
                        self.grid_details[neighbor] = 2
                    elif self.grid_details[neighbor] == 1:
                        self.grid_details[neighbor] = 2
                    elif self.grid_details[neighbor] == 2:
                        self.trigger_wave(neighbor)

    def register_damage(self, origin, target):
        """Registra daño en muros o puertas."""
        if origin in self.exit_positions and target in self.exit_positions:
            if not self.exit_positions[origin] and not self.exit_positions[target]:
                del self.exit_positions[origin]
                del self.exit_positions[target]
                origin_wall = list(self.grid_walls[origin][0])
                target_wall = list(self.grid_walls[target][0])
                path_org = self.direction(origin, target)
                path_targ = self.direction(target, origin)
                origin_wall[path_org] = "0"
                target_wall[path_targ] = "0"
                self.grid_walls[origin][0] = "".join(origin_wall)
                self.grid_walls[target][0] = "".join(target_wall)
                self.damage_counter += 1
        else:
            self.damage_counter += 1

    def direction(self, start, next):
        """Calcula la dirección de movimiento entre dos posiciones."""
        delta_x = next[0] - start[0]
        delta_y = next[1] - start[1]
        if delta_x == 0:
            if delta_y < 0:
                return 0
            else:
                return 2
        elif delta_y == 0:
            if delta_x < 0:
                return 1
            else:
                return 3

    def check_collision(self, start, next):
        """Verifica si hay una colisión entre dos posiciones."""
        direction = self.direction(start, next)
        wall_blocked = direction != None and self.grid_walls[start][0][direction] == "1"
        if start in self.exit_positions and next in self.exit_positions:
            doors_blocked = not (self.exit_positions[start] and self.exit_positions[next])
        else:
            doors_blocked = True
        return wall_blocked and doors_blocked

    def process_flashover(self):
        """Procesa la expansión de incendios y fantasmas."""
        smoke_cells = [pos for pos, val in self.grid_details.items() if val == 1]
        for smoke_cell in smoke_cells:
            neighbors = self.grid.get_neighborhood(smoke_cell, moore=False, include_center=False)
            for neighbor in neighbors:
                if self.grid_details[neighbor] == 2:
                    self.grid_details[smoke_cell] = 2
                    break
        for point in list(self.portraits):
            if self.grid_details[point] == 2:
                del self.portraits[point]
                self.casualties += 1
                break

    def update_simulation_status(self):
        """Actualiza el estado de la simulación."""
        if self.casualties >= 4 or self.damage_counter >= 24:
            self.simulation_status = "Defeat"
            return True
        elif self.rescued >= 7:
            self.simulation_status = "Victory"
            return True
        return False
    
    def print_schedule(self):
        print("Agentes en el Scheduler:")
        for agent in self.schedule.agents:
            print(f"Agente {agent.unique_id} con rol {agent.role} en posición {agent.pos}")

    def step(self):
        """Evoluciona un paso del modelo."""
        print(f"\n--- Turno {self.step_count} ---")
        self.datacollector.collect(self)  # Recolectar datos para análisis

        # Verificar si la simulación debe detenerse
        if self.update_simulation_status():
            print(f"[DEBUG] Estatus de la simulación: {self.simulation_status}")
            return

        self.step_count += 1
        print("[DEBUG] Iniciando pasos de los agentes en orden:")

        # Iterar sobre los agentes según su ID
        for agent in sorted(self.schedule.agents, key=lambda a: a.unique_id):
            agent.step()

        # Mostrar la energía restante de todos los agentes al final del turno
        print("\n[DEBUG] Energía de los agentes al final del turno:")
        for agent in sorted(self.schedule.agents, key=lambda a: a.unique_id):
            print(f"  - Agente {agent.unique_id} ({agent.role}): {agent.action_points} de energía.")
        
        # Realizar procesos adicionales del modelo
        self.spread_boos()
        self.process_flashover()
        self.add_portraits()
        self.update_simulation_status()


In [5]:
def procesar_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Procesar paredes
    WALLS = []
    index = 0
    for _ in range(6):  # 6 líneas de paredes
        row = []
        cells = lines[index].strip().split()
        for cell in cells:
            walls = [int(d) for d in cell]  # Cada celda se descompone en 4 dígitos
            row.append(walls)
        WALLS.append(row)
        index += 1

    # Procesar puntos de interés
    FAKE_ALARMS = []
    VICTIMS = []
    for _ in range(3):  # Máximo 3 líneas para puntos de interés
        r, c, marker_type = lines[index].strip().split()
        if marker_type == 'f':  # Falsa alarma
            FAKE_ALARMS.append((int(r), int(c)))
        elif marker_type == 'v':  # Víctima
            VICTIMS.append((int(r), int(c)))
        index += 1

    # Procesar marcadores de fuego
    FIRES = []
    for _ in range(10):  # 10 líneas de fuego
        r, c = map(int, lines[index].strip().split())
        FIRES.append((c, r))
        index += 1

    # Procesar marcadores de puertas
    DOORS = {}
    DOORS_CONNECTED = {}
    for _ in range(8):  # 8 líneas de puertas
        r1, c1, r2, c2 = map(int, lines[index].strip().split())
        r1, c1, r2, c2 = r1, c1, r2, c2
        DOORS[(r1, c1, r2, c2)] = (r1, c1, r2, c2)
        DOORS_CONNECTED[(r1, c1)] = (r2, c2)
        DOORS_CONNECTED[(r2, c2)] = (r1, c1)
        index += 1

    # Procesar puntos de entrada
    ENTRANCES = []
    for _ in range(4):  # 4 líneas de puntos de entrada
        r, c = map(int, lines[index].strip().split())
        ENTRANCES.append((r, c))
        index += 1

    return WALLS, FAKE_ALARMS, VICTIMS, FIRES, DOORS, DOORS_CONNECTED, ENTRANCES

# Ruta del archivo
file_path = './final.txt'

# Llamar a la función
WALLS, FAKE_ALARMS, VICTIMS, FIRES, DOORS, DOORS_CONNECTED, ENTRANCES = procesar_txt(file_path)

# Imprimir los resultados
print("Paredes:")
for row in WALLS:
    print(row)

print("\nFalsas alarmas:", FAKE_ALARMS)
print("\nVíctimas:", VICTIMS)
print("\nFuegos:", FIRES)
print("\nPuertas:", DOORS)
print("\nPuertas conectadas:", DOORS_CONNECTED)
print("\nEntradas:", ENTRANCES)

Paredes:
[[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [1, 1, 1, 1], [1, 1, 0, 1], [1, 1, 0, 1]]
[[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 1, 1], [0, 1, 0, 0], [0, 0, 0, 1]]
[[1, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [1, 1, 1, 0], [1, 0, 1, 1], [1, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]
[[0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 1, 0], [1, 0, 1, 1]]
[[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 1], [1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1]]
[[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 1]]

Falsas alarmas: [(2, 4)]

Víctimas: [(6, 7), (4, 8)]

Fuegos: [(7, 1), (3, 4), (2, 5), (5, 1), (2, 2), (6, 2), (1, 6), (8, 1), (1, 2), (4, 6)]

Puertas: {(1, 6, 1, 7): (1, 6, 1, 7), (2, 6, 2, 7): (2, 6, 2, 7), (3, 6, 4, 6): (3, 6, 4, 6), (4, 6, 4, 7): (4, 6, 4, 7), (4, 5, 5, 5): (4, 5, 5, 5), (4, 6, 5, 6): (4, 

In [69]:
# PARÁMETROS
WIDTH = 9
HEIGHT = 7
LUIGIS = 6

WALLS, FAKE_ALARMS, PORTRAITS, GHOSTS, DOORS, DOORS_CONNECTED, ENTRANCES = procesar_txt(file_path)

# Definir el número de simulaciones que quieres ejecutar
NUM_SIMULACIONES = 1

# Para almacenar los resultados de cada simulación
resultados_simulaciones = []

for sim in range(NUM_SIMULACIONES):
    print(f"\n--- Simulación {sim + 1} ---")
    model = MansionModel(LUIGIS, FAKE_ALARMS, PORTRAITS,WALLS, DOORS, GHOSTS, ENTRANCES)
    steps = 0

    while model.step_count <= 1000:
        model.step()
        steps += 1

        # Verificar si la simulación ha terminado
        if model.update_simulation_status():
            break

        # Mostrar la energía de cada Luigi al final de cada turno
        print(f"Turno {steps}: Energía de los luigis:")
        for agent in model.schedule.agents:
            if isinstance(agent, LuigiAgent):
                print(f"{agent.unique_id}: {agent.action_points} de energía")

    # Al finalizar, mostrar el estado final
    print(f"[DEBUG] La simulación ha terminado. Estatus: {model.simulation_status}")

    # Guardar resultados de la simulación actual
    resultado = {
        "simulacion": sim + 1,
        "steps": steps,
        # Changed the attributes to match the MansionModel class definition
        "damage": model.damage_counter,
        "total_deaths": model.losses,
        "saved_victims": model.rescued
    }
    resultados_simulaciones.append(resultado)

    # Mostrar los resultados de la simulación actual
    print(f"Simulación {sim + 1} Finalizada:")
    print(f"Number of steps: {steps}")
    # Changed the attributes to match the MansionModel class definition
    print(f"Damage: {model.damage_counter}")
    print(f"Deaths: {model.losses}")
    print(f"Saved Victims: {model.rescued}")
    if model.damage_counter >= 24:
        print("MANSION TAKEN OVER")
        print("GAME OVER")
    elif model.losses >= 4:
        print("4 DEATHS")
        print("GAME OVER")
    elif model.rescued >= 7:
        print("VICTORY!!!!")

# Mostrar un resumen de los resultados de todas las simulaciones
print("\n--- Resumen de todas las simulaciones ---")
for resultado in resultados_simulaciones:
    print(f"Simulación {resultado['simulacion']}:")
    print(f"  Steps: {resultado['steps']}")
    print(f"  Damage: {resultado['damage']}")
    print(f"  Deaths: {resultado['total_deaths']}")
    print(f"  Saved Victims: {resultado['saved_victims']}")


--- Simulación 1 ---
Agente 0 con rol rescuer colocado en posición (3, 1)
Agente 1 con rol firefighter colocado en posición (8, 1)
Agente 2 con rol rescuer colocado en posición (1, 5)
Agente 3 con rol firefighter colocado en posición (3, 6)
Agente 4 con rol rescuer colocado en posición (3, 1)
Agente 5 con rol firefighter colocado en posición (8, 1)

--- Turno 0 ---
[DEBUG] Iniciando pasos de los agentes en orden:

[DEBUG] Agente 0 (rescuer) inicia su turno en posición (3, 1). Energía inicial: 4.
[DEBUG] Agente 0 buscando retrato. Moviéndose hacia el retrato más cercano: (4, 2)
Agente 0 se mueve de (3, 1) a (4, 1).
[DEBUG] Agente 0 buscando retrato. Moviéndose hacia el retrato más cercano: (4, 2)
Agente 0 se mueve de (4, 1) a (4, 2).
Agente 0 encontró una falsa alarma en (4, 2). Eliminando de la lista y buscando otro retrato.
[DEBUG] Agente 0 buscando retrato. Moviéndose hacia el retrato más cercano: (8, 4)
Agente 0 se mueve de (4, 2) a (5, 2).
[DEBUG] Agente 0 buscando retrato. Movién